# Extending Buckaroo
Buckaroo is built for exploratory data analysis on unknown data.  Data in the wild is incredibly varied and so are the ways of visualizing it. Most table tools are built around allowing a single bespoke customization, with middle of the road defaults. Buckaroo takes a different approach. Buckaroo lets you build many highly specific configurations and then toggle between them quickly.  This makes it easier to build each configuration because you don't have to solve for every possibility.

This document walks you through how to add your own analysis to Buckaroo and allow users to toggle it

The extension points are
* [PluggableAnalysisFramework](https://buckaroo-data.readthedocs.io/en/latest/articles/pluggable.html) Used to add summary stats and column metadata for use by other steps
* [Styling](./styling-howto.ipynb) control the visual display of the table
* PostProcessing used to transform an entire dataframe
* AutoCleaning Automate transformations for dropping nulls, removing outliers and other pre-processing steps, cleans the dataframe and generates python code.  Not yet supported in 0.6

Each extension point is composable, and can be interactively mixed and matched

In [1]:
import pandas as pd
import numpy as np
from buckaroo.dataflow_traditional import StylingAnalysis
from buckaroo.pluggable_analysis_framework.pluggable_analysis_framework import ColAnalysis
import polars as pl
from polars import functions as F
from buckaroo.polars_buckaroo import PolarsBuckarooWidget

In [2]:
ROWS = 200
typed_df = pl.DataFrame({'int_col':np.random.randint(1,50, ROWS), 'float_col': np.random.randint(1,30, ROWS)/.7,
                         'timestamp':["2020-01-01 01:00Z", "2020-01-01 02:00Z", "2020-02-28 02:00Z", "2020-03-15 02:00Z", None] * 40,
                         "str_col": ["foobar", "Realllllly long string", "", None, "normal"]* 40})
typed_df = typed_df.with_columns(timestamp=pl.col('timestamp').str.to_datetime() )

In [ ]:
pbw = PolarsBuckarooWidget(typed_df)
pbw

# Adding a styling analysis
The `StylingAnalysis` class is used to control the display of a column based on the column metadata.  


Overriding the `config_from_column_metadata(col:str, sd:SingleColumnMetadata) -> ColumnConfig` computes the config for a single column give that column's metadata.

This lets you customize based on metadata collected about a column.  This works with the [PluggableAnalysisFramework](https://buckaroo-data.readthedocs.io/en/latest/articles/pluggable.html),  you can specify required fields that are necessary.  Adding requirements like this garuntees that errors are spotted early.

StylingAnalysis works for both Polars and Pandas because it only receives a dictionary with simple python values

In [ ]:
from typing import Any

class EverythingStyling(StylingAnalysis):
    """
    This styling shows as much detail as possible
    """
    df_display_name = "everything"

    pinned_rows = [{'primary_key_val': 'dtype', 'displayer_args': {'displayer': 'obj' }}]


    #Styling analysis handles column iteration for us.
    
    #the type should be
    #def style_column(col:str, column_metadata: SingleColumnMetadata) -> ColumnConfig:
    @classmethod
    def style_column(kls, col:str, column_metadata: Any) -> Any:
        digits = 10
        if column_metadata['is_integer']:
            disp = {'displayer': 'float', 'min_fraction_digits':0, 'max_fraction_digits':0}
        elif column_metadata['is_numeric']:
            disp = {'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}
        # FIXME, because we don't have a DataFrame library agnostic way of saying "is_string" 
        # this styling analysis will only work with polars
        elif column_metadata['dtype'] == pl.String:
            disp = {'displayer': 'string'}
        elif column_metadata['dtype'] == pl.Datetime:
            disp =  {'displayer': 'datetimeDefault'}
        else:
            disp = {'displayer': 'obj'}
        return {'col_name':col, 'displayer_args': disp }

class AbrevStyling(StylingAnalysis):
    """
    This styling shows as much detail as possible
    """
    df_display_name = "Abrev"

    pinned_rows = [{'primary_key_val': 'dtype', 'displayer_args': {'displayer': 'obj'}}]

    @classmethod
    def style_column(kls, col:str, column_metadata: Any) -> Any:
        digits = 3
        if column_metadata['is_integer']:
            disp = {'displayer': 'float', 'min_fraction_digits':0, 'max_fraction_digits':0}
        elif column_metadata['is_numeric']:
            disp = {'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}
        elif column_metadata['dtype'] == pl.Datetime:
            disp = {'displayer': 'datetimeLocaleString','locale': 'en-US',  'args': {}}
        elif column_metadata['dtype'] == pl.String:
            disp = {'displayer': 'string', 'max_length':15}
        else:
            disp = {'displayer': 'obj'}
        return {'col_name':col, 'displayer_args': disp }

sbw = PolarsBuckarooWidget(typed_df)
sbw.add_analysis(EverythingStyling)
sbw.add_analysis(AbrevStyling)
sbw

https://github.com/paddymul/buckaroo/blob/main/tests/unit/polars_analysis_management_test.py
https://github.com/paddymul/buckaroo/blob/main/tests/unit/polars_analysis_management_test.py
https://github.com/paddymul/buckaroo/blob/main/buckaroo/customizations/polars_analysis.py


In [ ]:
from buckaroo.pluggable_analysis_framework.utils import (json_postfix)
import polars.selectors as cs
class Quin99Analysis(StylingAnalysis):
    select_clauses = [
        cs.numeric().quantile(.99).name.map(json_postfix('quin99'))]
    pinned_rows = [{'primary_key_val': 'quin99', 'displayer_args': {'displayer': 'obj' }}]
    df_display_name = 'quin99'
    data_key = "empty"  # the non pinned rows will pull from the empty dataframe

sbw = PolarsBuckarooWidget(typed_df)
sbw.add_analysis(Quin99Analysis)
sbw

Let's look at pinned_rows, they can be modified by setting `pinned_rows` on Buckaroo Instaniation

# lets add a post processing method

In [ ]:
from polars import functions as F
from buckaroo.pluggable_analysis_framework.polars_analysis_management import PolarsAnalysis

In [ ]:
#typed_df.select(F.all(), pl.col('float_col').lt(5).replace(True, "foo").replace(False, None).alias('errored_float'))[:3]

In [3]:
bw = PolarsBuckarooWidget(typed_df)

class TransposeProcessing(ColAnalysis):
    @classmethod
    def post_process_df(kls, df):
        return [df.transpose(), {}]
    post_processing_method = "transpose"
bw.add_analysis(TransposeProcessing)
bw

PolarsBuckarooWidget(buckaroo_options={'sampled': ['random'], 'auto_clean': ['aggressive', 'conservative'], 'p…

In [9]:
# that's a lot to type to just transpose, let's make this cleaner
def add_processing(bw_instance):
    def proc_func_decorator(processing_func):
        
        class DecoratedProcessing(ColAnalysis):
            @classmethod
            def post_process_df(kls, df):
                new_df = processing_func(df)
                return [new_df, {}]
            post_processing_method = processing_func.__name__
        bw_instance.add_analysis(DecoratedProcessing)
        
    return proc_func_decorator

In [10]:
bw = PolarsBuckarooWidget(typed_df)
@add_processing(bw)
def transpose(df):
    return df.transpose()

bw

PolarsBuckarooWidget(buckaroo_options={'sampled': ['random'], 'auto_clean': ['aggressive', 'conservative'], 'p…

In [ ]:
class ValueCountPostProcessing(PolarsAnalysis):
    @classmethod
    def post_process_df(kls, df):
        result_df = df.select(
            F.all().value_counts().implode().list.gather(pl.arange(0, 10), null_on_oob=True).explode().struct.rename_fields(['val', 'unused_count']).struct.field('val').prefix('val_'),
            F.all().value_counts().implode().list.gather(pl.arange(0, 10), null_on_oob=True).explode().struct.field('count').prefix('count_'))
        return [result_df, {}]
    post_processing_method = "value_counts"
    

class TransposeProcessing(ColAnalysis):
    @classmethod
    def post_process_df(kls, df):
        return [df.transpose(), {}]
    post_processing_method = "transpose"

class ShowErrorsPostProcessing(PolarsAnalysis):
    @classmethod
    def post_process_df(kls, df):
        result_df = df.select(
            F.all(),
            pl.col('float_col').lt(20).replace(True, "foo").replace(False, None).alias('errored_float'))
        extra_column_config = {
            'float_col': {'column_config_override': {
                'color_map_config': {
                    'color_rule': 'color_not_null',
                    'conditional_color': 'red',
                    'exist_column': 'errored_float'},
                'tooltip_config': { 'tooltip_type':'simple', 'val_column': 'errored_float'}}},
            'errored_float': {'column_config_override': {'merge_rule': 'hidden'}}}
        return (result_df, extra_column_config)
    post_processing_method = "show_errors"

# In this case we are going to extend PolarsBuckarooWidget so we can take this combination with us
base_a_klasses = PolarsBuckarooWidget.analysis_klasses.copy()
base_a_klasses.extend([ValueCountPostProcessing, 
                       TransposeProcessing, 
                       ShowErrorsPostProcessing])
class VCBuckarooWidget(PolarsBuckarooWidget):
    analysis_klasses = base_a_klasses
vcb = VCBuckarooWidget(typed_df, debug=False)
vcb

## Where to use PostProcessing
Post processing functions are no argument transformations.  I can't think of a lot of generic whole dataframe operations.

`ValueCount` and `Transpose` are geenric.  `ShowErrors` depends on two specific columns.

I expect Post processing to be very useful for small custom apps built on top of Buckaroo.  When you know the columns and you want a strict set of transforms, PostProcessing is a great fit

# Putting it all together

You can compose (combine) the PluggableAnalysisFramework, PostProcessing and Styling into a single widget.  And you can manipulate PostProcessing separately from Styling.

In [ ]:
from buckaroo.customizations.polars_analysis import (
    VCAnalysis, BasicAnalysis, PlTyping,
    PlColDisplayHints, HistogramAnalysis, ComputedDefaultSummaryStats)
from buckaroo.customizations.styling import DefaultSummaryStatsStyling, DefaultMainStyling

class KitchenSinkWidget(PolarsBuckarooWidget):
    #let's be explicit here and show all of the built in analysis klasses
    analysis_klasses = [
    # The default analysis methods for Polars
    VCAnalysis, BasicAnalysis, PlTyping, PlColDisplayHints,
    HistogramAnalysis, ComputedDefaultSummaryStats,
    # default buckaroo styling
    DefaultSummaryStatsStyling, DefaultMainStyling,
        
    # our Quin99 analysis
    Quin99Analysis,  # adds a styling method
    #our PostProcessing classes
    ValueCountPostProcessing, TransposeProcessing, ShowErrorsPostProcessing,
    #our styling methods
    EverythingStyling, AbrevStyling]
ksw = KitchenSinkWidget(typed_df)
ksw

In [ ]:
list(ksw.df_display_args.keys())

# What about autocleaning and the low code UI

Auto cleaning and the low code UI work together for more fine grained editting of data.  The low code UI presents a gui that works on columns and allows functions with arguments.  

Auto cleaning works to suggest operations that are then loaded into the low code ui.  Then these operations can be editted or removed.
Auto cleaning options can be cycled through to generate different cleanings.

## Why did this release remove auto cleaning and the low code UI?

Although auto cleaning and the low code UI is my favorite feature of Buckaroo, and the first part I built, it hasn't seemed to have gained traction with users.  Buckaroo for that matter hasn't gained a lot of traction.  For the time being I have decided to put more effort into refining and promoting the parts of Buckaroo that people do understand.  

# How to develop your own library

Just start writing fun ctions and customizations.  Don't worry about making it reusable at first.  If you have to write a variation, hard code it.  Later you can come back and refactor the functions.
You can quickly see the result of a functiom.

Buckaroo is built around interactive development.  Not just an interactive table widget, but interactive development.  That's because working with data is an interactive pursuit, jupyter is a rich fully featured environment and buckaroo works with it.  

With interactive development you can start quickly without having to setup a development environemnt.

In [ ]:
class AdaptingStylingAnalysis(SimpleStylingAnalysis):
    requires_summary = ["histogram", "is_numeric", "dtype", "is_integer"]
    pinned_rows = [
        obj_('dtype'),
        {'primary_key_val': 'histogram', 'displayer_args': { 'displayer': 'histogram' }}]
    @classmethod
    def style_column(kls, col, sd):
        #print(col, list(sd.keys()))
        if len(sd.keys()) == 0:
            #I'm still having problems with index and polars
            return {'col_name':col, 'displayer_args': {'displayer': 'obj'}}
        digits = 3
        if sd['is_integer']:
            disp = {'displayer': 'float', 'min_fraction_digits':0, 'max_fraction_digits':0}
        elif sd['is_numeric']:
            disp = {'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}
        else:
            disp = {'displayer': 'obj'}
        return {'col_name':col, 'displayer_args': disp }


base_a_klasses = PolarsBuckarooWidget.analysis_klasses.copy()
base_a_klasses.extend([AdaptingStylingAnalysis, ValueCountPostProcessing])
class ABuckarooWidget(PolarsBuckarooWidget):
    analysis_klasses = base_a_klasses
acb = ABuckarooWidget(typed_df)
acb